In [7]:
%load_ext autoreload
%autoreload 2
import os
os.chdir('/vepfs/fs_users/yftc/code/CureALL')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from unicore.data import (
    Dictionary,
    NestedDictionaryDataset,
    AppendTokenDataset,
    PrependTokenDataset,
    RightPadDataset,
    EpochShuffleDataset,
    TokenizeDataset,
    RawLabelDataset,
    RightPadDataset2D,
    FromNumpyDataset,
    RawArrayDataset,
)
from cureall.data.components.atom_type_dataset import AtomTypeDataset
from cureall.data.components.coord_pad_dataset import RightPadDatasetCoord
from cureall.data.components.cropping_dataset import CroppingDataset
from cureall.data.components.distance_dataset import DistanceDataset, EdgeTypeDataset
from cureall.data.components.normalize_dataset import NormalizeDataset
from cureall.data.components.conformer_sample_dataset import ConformerSampleDataset
from cureall.data.components.key_dataset import KeyDataset
from cureall.data.components.remove_hydrogen_dataset import RemoveHydrogenDataset
from cureall.data.components.lmdb_dataset import LMDBDataset

from cureall.data.components.base_datasets import FromNumpyDataset, TTADataset
from cureall.data.components.uce_datasets import UCEBaseDataset
from cureall.data.components.unimol_tokenizer import UniMolTokenizer

In [10]:
from cureall.data.components import UniMolTokenizeDataset, UniMolTokenizer

tokenizer = UniMolTokenizer()

In [57]:
raw_dataset = LMDBDataset("/vepfs/fs_users/yftc/code/CureALL/datas/lincs/val.lmdb")
# raw_dataset = Add2DConformerDataset(raw_dataset, "smi", "atoms", "coordinates")
smi_dataset = KeyDataset(raw_dataset, "smi")

# train
sampled_dataset = ConformerSampleDataset(raw_dataset, 42, "atoms", "coordinates")
# val
# sampled_dataset = TTADataset(raw_dataset, 42, "atoms", "coordinates", 10)

dataset = AtomTypeDataset(raw_dataset, sampled_dataset)
dataset = RemoveHydrogenDataset(dataset, "atoms", "coordinates", True, True)

dataset.__len__()

9693

In [54]:
import torch
raw_dataset[0].keys()
raw_dataset[2]['cell_batch'][1]
total_sum = torch.sum(raw_dataset[2]['cell_batch'][1])
total_sum

tensor(1071.)

In [5]:
dataset = CroppingDataset(dataset, 42, "atoms", "coordinates", 512)
dataset = NormalizeDataset(dataset, "coordinates", normalize_coord=True)
src_dataset = KeyDataset(dataset, "atoms")
src_dataset = UniMolTokenizeDataset(src_dataset, tokenizer, max_seq_len=512)
coord_dataset = KeyDataset(dataset, "coordinates")

In [6]:
def PrependAndAppend(dataset, pre_token, app_token):
    dataset = PrependTokenDataset(dataset, pre_token)
    return AppendTokenDataset(dataset, app_token)


src_dataset = PrependAndAppend(src_dataset, tokenizer.cls_token_id, tokenizer.sep_token_id)
edge_type = EdgeTypeDataset(src_dataset, len(tokenizer))

In [7]:
from cureall.data.components import UCEBaseDataset
uce_dataset = UCEBaseDataset(raw_dataset)

In [8]:
uce_dataset[0]

(tensor([[     3, 144156,  18105,  ...,      0,      0,      0]]),
 tensor([[1., 1., 1.,  ..., 0., 0., 0.]]),
 41,
 1071,
 tensor([[3.0000e+00, 1.4416e+05, 1.8105e+04,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00]]))

In [9]:
coord_dataset = FromNumpyDataset(coord_dataset)
coord_dataset = PrependAndAppend(coord_dataset, 0.0, 0.0)
distance_dataset = DistanceDataset(coord_dataset)

nest_dataset = NestedDictionaryDataset(
    {
        "net_input": {
            "src_tokens": RightPadDataset(
                src_dataset,
                pad_idx=tokenizer.pad_token_id,
            ),
            "src_coord": RightPadDatasetCoord(
                coord_dataset,
                pad_idx=0,
            ),
            "src_distance": RightPadDataset2D(
                distance_dataset,
                pad_idx=0,
            ),
            "src_edge_type": RightPadDataset2D(
                edge_type,
                pad_idx=0,
            ),
            "uce_batches": uce_dataset,
            
        },
        "smi_name": RawArrayDataset(smi_dataset),
    }
)

In [10]:
from torch.utils.data import DataLoader

dataloader = DataLoader(nest_dataset, batch_size=2, num_workers=0, collate_fn=nest_dataset.collater)

In [18]:
for item in dataloader:
    item = item
    print(item)
    break

OrderedDict([('net_input', OrderedDict([('src_tokens', tensor([[1, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         5, 4, 6, 4, 4, 4, 4, 5, 4, 6, 5, 4, 4, 4, 4, 4, 4, 5, 6, 6, 2, 0, 0, 0],
        [1, 4, 6, 4, 6, 4, 4, 4, 4, 4, 4, 4, 6, 4, 4, 4, 4, 5, 4, 6, 4, 4, 4, 4,
         4, 9, 4, 4, 4, 4, 6, 5, 4, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])), ('src_coord', tensor([[[ 0.0000,  0.0000,  0.0000],
         [-2.7698, -3.3711, -2.0201],
         [-2.5422, -3.5812, -0.5955],
         [-2.9577, -4.8816, -0.0633],
         [-4.4616, -5.0266, -0.0088],
         [-5.1330, -5.8453, -0.9290],
         [-6.5219, -5.9746, -0.8804],
         [-7.2557, -5.2918,  0.0889],
         [-6.5996, -4.4686,  1.0054],
         [-5.2117, -4.3345,  0.9567],
         [-2.0857, -2.6121,  0.2953],
         [-1.9946, -2.8338,  1.5090],
         [-1.7784, -1.1918, -0.2411],
         [-3.1086, -0.4329, -0.3995],
         [-2.9519,  1.0258, -0.7711],
         [-2.2915,  1.4047, -1.9

In [19]:
item["net_input"]["uce_batches"]

(tensor([[3.0000e+00, 1.4416e+05, 1.8105e+04,  ..., 1.6360e+04, 1.7073e+04,
          2.0000e+00],
         [3.0000e+00, 1.4416e+05, 1.9996e+04,  ..., 2.6424e+04, 1.3992e+04,
          2.0000e+00]]),
 tensor([[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]))

In [12]:
item["net_input"]["src_distance"].shape, item["net_input"]["src_edge_type"].shape, item["net_input"][
    "src_coord"
].shape, item["net_input"]["src_tokens"].shape

(torch.Size([2, 48, 48]),
 torch.Size([2, 48, 48]),
 torch.Size([2, 48, 3]),
 torch.Size([2, 48]))

In [13]:
from cureall.models import UniMolModel, UniMolConfig

In [14]:
import yaml

config = yaml.safe_load(open("/home/wangxi/develop/cln/CureALL/configs/model/unimol_only.yaml"))

In [15]:
configs = UniMolConfig(**config)

In [16]:
model = UniMolModel(configs)

In [17]:
model(**item["net_input"], features_only=True)[0].shape

torch.Size([2, 48, 512])

In [25]:
from cureall.models import UCEConfig, UCEModel
config = UCEConfig(pretrained_model_name_or_path="/home/wangxi/develop/cln/UCE/model_files/33l_8ep_1024t_1280.torch")
uce_model = UCEModel(config)

In [29]:
uce_model(item["net_input"]["uce_batches"][0], item["net_input"]["uce_batches"][1])

tensor([[-0.0414,  0.0028,  0.0113,  ...,  0.0067, -0.0247, -0.0405],
        [-0.0161, -0.0215,  0.0104,  ..., -0.0109, -0.0413, -0.0269]],
       grad_fn=<DivBackward0>)